In [ ]:
!curl -r 0-304857599 -o hi.txt https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/v1-indiccorp/hi.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  290M  100  290M    0     0  38.5M      0  0:00:07  0:00:07 --:--:-- 44.1M


In [ ]:
!pip install transformers torch torchvision torchaudio indic-nlp-library nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 20.1 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19188 sha256=4c26950f7dc2b11da9a3ce927e12d4fad1d065824f1f38f880ad52acc9f6e53e
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [ ]:
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT").cuda()
print(len(tokenizer))
tokenizer.add_special_tokens({"sep_token": "<|sep|>", "bos_token":'<|startoftext|>',
                                           "pad_token":'<|pad|>'})
print(tokenizer.all_special_tokens)
model.resize_token_embeddings(len(tokenizer))
print_gpu_utilization()

100000
['<|startoftext|>', '<|endoftext|>', '<|sep|>', '<|pad|>']
GPU memory occupied: 6667 MB.


In [ ]:
from typing import Sequence, List
from random import shuffle, randint, choice
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize
stop_words = {"पर",
"इन",
"वह",
"यिह",
"वुह",
"जिन्हें",
"जिन्हों",
"तिन्हें",
"तिन्हों",
"किन्हों",
"किन्हें",
"इत्यादि",
"द्वारा",
"इन्हें",
"इन्हों",
"उन्हों",
"बिलकुल",
"निहायत",
"ऱ्वासा",
"इन्हीं",
"उन्हीं",
"उन्हें",
"इसमें",
"जितना",
"दुसरा",
"कितना",
"दबारा",
"साबुत",
"वग़ैरह",
"दूसरे",
"कौनसा",
"लेकिन",
"होता",
"करने",
"किया",
"लिये",
"अपने",
"नहीं",
"दिया",
"इसका",
"करना",
"वाले",
"सकते",
"इसके",
"सबसे",
"होने",
"करते",
"बहुत",
"वर्ग",
"करें",
"होती",
"अपनी",
"उनके",
"कहते",
"होते",
"करता",
"उनकी",
"इसकी",
"सकता",
"रखें",
"अपना",
"उसके",
"जिसे",
"तिसे",
"किसे",
"किसी",
"काफ़ी",
"पहले",
"नीचे",
"बाला",
"यहाँ",
"जैसा",
"जैसे",
"मानो",
"अंदर",
"भीतर",
"पूरा",
"सारा",
"होना",
"उनको",
"वहाँ",
"वहीं",
"जहाँ",
"जीधर",
"उनका",
"इनका",
"के",
"हैं",
"गया",
"बनी",
"एवं",
"हुआ",
"साथ",
"बाद",
"लिए",
"कुछ",
"कहा",
"यदि",
"हुई",
"इसे",
"हुए",
"अभी",
"सभी",
"कुल",
"रहा",
"रहे",
"इसी",
"उसे",
"जिस",
"जिन",
"तिस",
"तिन",
"कौन",
"किस",
"कोई",
"ऐसे",
"तरह",
"किर",
"साभ",
"संग",
"यही",
"बही",
"उसी",
"फिर",
"मगर",
"का",
"एक",
"यह",
"से",
"को",
"इस",
"कि",
"जो",
"कर",
"मे",
"ने",
"तो",
"ही",
"या",
"हो",
"था",
"तक",
"आप",
"ये",
"थे",
"दो",
"वे",
"थी",
"जा",
"ना",
"उस",
"एस",
"पे",
"उन",
"सो",
"भी",
"और",
"घर",
"तब",
"जब",
"अत",
"व",
"न"}

def swap_random(a: List, exclude: List):
    if len(a) <= 1:
        return
    for _ in range(5):

        i, j = randint(0, len(a) - 1), randint(0, len(a) - 1)

        if a[i] in exclude or a[j] in exclude:
            continue
        a[i], a[j] = a[j], a[i]
        break

def corrupt(tokens: Sequence[str]) -> Sequence[str]:
    all_tokens = tokens
    tokens = [token for token in tokens if token not in stop_words]
    if len(tokens) == 0:
        tokens = all_tokens # if all words are stop words restore
    i = randint(1, len(tokens)//3+1)
    for _ in range(i):
        swap_random(tokens, {"!", "?"})
    return tokens

def load_data():
    sentences = []
    normalizer_factory = IndicNormalizerFactory()
    normalizer = normalizer_factory.get_normalizer("hi",remove_nuktas=False)
    with open("hi.txt") as f:
        for _, line in zip(range(2000), f):
            sentences.append(indic_tokenize.trivial_tokenize(normalizer.normalize(line.strip())))
    return sentences

for s in load_data():
    print(s)


['आवेदन', 'करने', 'की', 'आखिरी', 'तारीख', '31', 'जनवरी', ',', '2020', 'है', '।']
['इतनी', 'दुआ', 'कर', 'दो', 'हमारे', 'लिए', 'कि', 'जितना', 'प्यार', 'दुनिया', 'ने', 'आपको', 'दिया', 'है', ',', 'बस', 'उतना', 'ही', 'हमें', 'भी', 'मिल', 'जाए', '।', '"']
['मोदी', 'सरकार', 'के', 'पहले', 'कार्यकाल', 'में', 'भी', 'तीन', 'तलाक', 'को', 'लेकर', 'बिल', 'लाया', 'गया', 'था', ',', 'हालांकि', 'तब', 'यह', 'राज्यसभा', 'में', 'पास', 'नहीं', 'हो', 'पाया', 'था', '.']
['भाजपा', 'के', 'दिवंगत', 'नेता', 'प्रमोद', 'महाजन', 'की', 'बेटी', 'पूनम', 'महाजन', 'को', 'सचिव', 'बनाया', 'गया', 'है', '.']
['ऐसी', 'स्थिति', 'में', 'एक', 'न्यायपूर्ण', 'सरकार', 'सार्वजनिक', 'वित्त', 'का', 'इस', 'तरह', 'इस्तेमाल', 'करती', 'है', 'कि', 'संसाधनों', 'का', 'आवंटन', ',', 'सभी', 'के', 'उपभोग', 'वाले', 'उत्पादों', 'की', 'व्यवहार्यता', 'और', 'समग्र', 'वृहद', '-', 'आर्थिक', 'प्रबंधन', "'", 'निष्पक्षता', 'के', 'रूप', 'में', 'न्याय', "'", 'को', 'बढ़ाए', '।']
['दिलचस्प', 'है', 'कि', 'डीसीएचएल', 'के', 'चेयरमैन', 'टी', 'वेंकटरमन', 'रेड्डी',

In [ ]:
from copy import deepcopy
import torch
from torch.utils.data import Dataset, random_split

IGNORE_INDEX = -100

class ReconstructionDataset:
  def __init__(self, max_length=75):
    sentences = load_data()
    self.input_ids = []
    self.attn_masks = []
    self.labels = []
    for sentence in sentences:
        corrupted = corrupt(sentence)
        print(sentence)
        print(corrupted)
        print()
        sentence_ignore_len = len(tokenizer('<|startoftext|>' + " ".join(corrupted) + "<|sep|>")["input_ids"])
        encodings_dict = tokenizer('<|startoftext|>' + " ".join(corrupted) + "<|sep|>" + " ".join(sentence) + '<|endoftext|>', truncation=True,
                                    max_length=max_length, padding="max_length", add_special_tokens=True)
        input_ids = torch.tensor(encodings_dict['input_ids'])
        self.input_ids.append(input_ids)
        label = deepcopy(input_ids)
        label[:sentence_ignore_len] = IGNORE_INDEX
        self.labels.append(label)

  def __len__(self):
      return len(self.input_ids)

  def __getitem__(self, idx):
      return self.input_ids[idx], self.labels[idx]

dataset = ReconstructionDataset()
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

Streaming output truncated to the last 5000 lines.
['मेरठ', 'बारे', 'में', 'ऐसी', 'घटना', 'है', 'याद', 'आज', 'रोंगटे', 'खड़े', 'जाते', '।', 'बात', '1987', 'की', 'मना', '।', 'मेरठ', 'पत्रकारों', 'दंगा', 'कर्फ्यू', 'लग', 'आ', 'कर्फ्यू', 'लगे', 'दिन', 'गए', '।', 'डी', '.', 'एम', '.', 'केवल', 'स्थानीय', 'संपादकों', 'पास', 'दिए', ',', 'बाकी', 'पत्रकारों', 'पास', 'देने', 'है', '।', 'विचार', 'मैंने', 'में', 'आफिस', 'बुलाया', 'विचार', 'जागरण', 'अभय', 'गुप्ता', 'ओ', '.', 'पी', '.', 'सक्सेना', ',', 'अमर', 'उजाला', 'सुनील', 'छइंया', 'हरि', 'जोशी', 'अलावा', 'हिंदुस्तान', 'पत्रकार', 'जिनका', 'नाम', 'याद', '।', ',', 'अलावा', 'चार', 'पांच', 'अन्य', 'पत्रकार', 'समस्या', '।', 'सिर', 'जोड़', 'बैठे', '।']

['इस', 'दिशा', 'में', 'एक', 'कदम', 'बढ़ते', 'हुए', 'मुख्यमंत्री', 'योगी', 'आदित्यनाथ', 'के', 'आधिकारिक', 'ट्विटर', 'अकाउंट', 'पर', 'शुक्रवार', 'को', 'इस', 'संबंध', 'में', 'जानकारी', 'साझा', 'की', 'गई', '।', 'ट्वीट', 'में', 'कहा', 'गया', 'है', 'कि', 'इसके', 'तहत', 'मेक', 'इन', 'यूपी', 'विभाग', 'की', 'स्

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print_gpu_utilization()

GPU memory occupied: 5885 MB.


In [ ]:
training_args = TrainingArguments(learning_rate=5e-6, output_dir='./results', num_train_epochs=3, logging_steps=10, save_steps=5000,optim="adafactor",
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,gradient_accumulation_steps=4, gradient_checkpointing=True,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                'labels': torch.stack([f[1] for f in data])})
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,6.611300
20,5.434900
30,2.472200
40,2.161800
50,1.110500
60,2.674700
70,5.837700
80,1.211100
90,1.284800
100,1.826300


TrainOutput(global_step=1350, training_loss=1.3479221605371545, metrics={'train_runtime': 3113.0893, 'train_samples_per_second': 1.735, 'train_steps_per_second': 0.434, 'total_flos': 2936904376320000.0, 'train_loss': 1.3479221605371545, 'epoch': 3.0})

In [ ]:
def print_paraphrases(sentence):
    normalizer_factory = IndicNormalizerFactory()
    normalizer = normalizer_factory.get_normalizer("hi",remove_nuktas=False)
    corrupted = " ".join(corrupt(indic_tokenize.trivial_tokenize(normalizer.normalize(sentence.strip()))))
    corrupt_in = "<|startoftext|>" + corrupted + "<|sep|>"
    generated = tokenizer(corrupt_in, return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(generated, do_sample=True, top_k=20,
        top_p=0.975, temperature=1.0,
        no_repeat_ngram_size=10, num_return_sequences=20)
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}".format(i, tokenizer.decode(sample_output).split("<|sep|>")[1].split("<|endoftext|>")[0]))

In [ ]:
print_paraphrases("मैं फल खाता हूँ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0:  जब मैं नहीं ख
1:  मैं खाता ह
2:  मैं खाता नह
3:  मैं तुम्ह
4:  अब मैं नहीं ख
5:  जब मैं ही नह
6:  खाता नहीं ह
7:  ये फल मैं ह
8:  खाता फल म
9:  उनके खाते
10:  तब मैं उसे फ
11:  और किसका
12:  खाता नहीं म
13:  खाता मैं ह
14:  फल मैं खात
15:  खाता को फ
16:  अपने घर मैं ह
17:  मैं खाता ह
18:  जब मैं हूँ
19:  इसलिए मैं फ


In [ ]:
model.save_pretrained("paraphraser-hindi")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip -r paraphraser-hindi.zip ./paraphraser-hindi

  adding: paraphraser-hindi/ (stored 0%)
  adding: paraphraser-hindi/generation_config.json (deflated 24%)
  adding: paraphraser-hindi/config.json (deflated 50%)
  adding: paraphraser-hindi/pytorch_model.bin (deflated 9%)


In [ ]:
# mount it
from google.colab import drive
drive.mount('/content/drive')
# copy it there


Mounted at /content/drive


In [ ]:
!cp paraphraser-hindi.zip /content/drive/MyDrive